In [1]:
import pandas as pd
import numpy as np

#from scipy.spatial import procrustes

import matplotlib.pyplot as plt

from joblib import load
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut

In [2]:
# Laden der Input-Features, Konvertierung zu ndarray

df = pd.read_csv("results_facemesh.csv", dtype={"relative_filepath": str, "facs_str": str})
df = df[~df.l_1_x.isnull()]
df["study_participant"] = df.relative_filepath.apply(lambda fpath: int(fpath[:3]))
X_all = df[[*[col for col in df.columns if "_x" in col or "_y" in col or "_z" in col]]].values

X_norm = np.loadtxt('X_norm.txt')
pain_present = np.loadtxt('pain_present.txt')
mean_faces = np.loadtxt('mean_faces.txt')

In [ ]:
%matplotlib inline

# Laden der Modelle und Plots

X_all_mean = np.hstack([mean_faces, X_all])
X_norm_mean = np.hstack([mean_faces, X_norm])

cv = LeaveOneGroupOut()

expand_features = True
class_weights = "_scaledPSPI"
eval_metric = "auc"  # "aucpr"
model_name_fmt = (f"models/models_pain_classifier/pain_mean_norm_fishyFalse"
                  f"_expand{expand_features}{class_weights}"
                  f"_{eval_metric}_fold{{}}.joblib")
#fig, axarr = plt.subplots(nrows=5, ncols=5, figsize=(15 ,15), sharex=True, sharey=True)
for i, (train, test) in enumerate(cv.split(X_all_mean, pain_present, groups=df.study_participant)):
    model_fname = model_name_fmt.format(i)
  #  ax = axarr[i // 5, i % 5]
    
    
#X_train, X_test, y_train, y_test = train_test_split(X_all_mean, pain_present, test_size=0.2, random_state=42)
#X_train2, X_test2, y_train2, y_test2 = train_test_split(X_norm_mean, pain_present, test_size=0.2, random_state=42)

pipe1 = load('models/pain_mean_procrustes.joblib')
pipe2 = load('models/au9.joblib')
pipe3 = load('models/pain_mean_norm.joblib')
    
y_pred1 = pipe1.predict(X_all_mean[test])
y_pred2 = pipe2.predict(X_all_mean[test])
y_pred3 = pipe3.predict(X_norm_mean[test])
y_true = pain_present[test]
    
plot_confusion_matrix(pipe1, X_all_mean[test], pain_present[test])
plot_confusion_matrix(pipe2, X_all_mean[test], pain_present[test])
plot_confusion_matrix(pipe3, X_norm_mean[test], pain_present[test])

print(classification_report(y_true, y_pred1, target_names = None))
print(classification_report(y_true, y_pred2, target_names = None))
print(classification_report(y_true, y_pred3, target_names = None))

print(pipe1.score(X_all_mean[test], pain_present[test]) )
print(pipe2.score(X_all_mean[test], pain_present[test]))
print(pipe3.score(X_norm_mean[test], pain_present[test]))

In [ ]:
#from scikitplot.metrics import plot_roc_curve
from sklearn.metrics import plot_roc_curve, auc

pain2=pain_present.astype(int)
plot_roc_curve(pipe,X_all_mean[test], pain_present[test])
                        #name=f"LOGO subj. {subjects[i]} fold {i}",
                        #alpha=0.6, lw=1, ax=ax)

                    

In [13]:
print(pipe1.score(X_all_mean[test], pain_present[test]) )
print(pipe2.score(X_all_mean[test], pain_present[test]))
print(pipe3.score(X_norm_mean[test], pain_present[test]))

0.9929203539823008
0.41238938053097346
0.6837758112094395
